# Introduction
The goal of this notebook is to examine RLCT estimation in 2D for three toy synthetic datasets. We will compare the performance of implicit variational inference versus expliciit variational inference. 

First we set up the parameters which feed into our training function.

In [ ]:
# %load_ext autoreload
# %autoreload 2

from __future__ import print_function

from torch.distributions.uniform import Uniform
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal

import sys
sys.path.append('../')
from main import *

class Args:
    
    syntheticsamplesize = 500
    batchsize = 100
    w_dim = 2
    dpower = None
    posterior_viz = True
    sanity_check = True

    epochs = 200
    prior = 'gaussian'
    pretrainDepochs = 100
    trainDepochs = 50
    n_hidden_D = 128
    num_hidden_layers_D = 1
    n_hidden_G = 128
    num_hidden_layers_G = 1

    lr_primal = 1e-3
    lr_dual = 1e-3
    lr = 1e-2

    beta_auto_liberal = False
    beta_auto_conservative = False
    beta_auto_oracle = False
    betasbegin = 0.1
    betasend = 1.5
    betalogscale = True
    numbetas = 5

    elasticnet_alpha = 0.5
    R = 200

    cuda = False

    log_interval = 50
    notebook = True
    
args = Args()
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}


The train function draws a dataset, trains both explicit and implicit variational inference. Results are logged as posterior graphs and RLCT least squares plot. 

In [ ]:
def train(args):

    # draw new training-testing split
    train_loader, valid_loader, test_loader = get_dataset_by_id(args, kwargs)

    # get a grid of inverse temperatures [beta_1/log n, \ldots, beta_k/log n]
    set_betas(args)

    mc = 1
    saveimgpath = None
    nll_betas_explicit = np.empty(0)
    nll_betas_implicit = np.empty(0)

    for beta_index in range(args.betas.shape[0]):

        # train explicit variational inference
        print('Begin training EVI')
        var_model = train_explicitVI(train_loader, valid_loader, args, mc, beta_index, True, saveimgpath)
        print('Finished training EVI')
        nllw_array_explicit = approxinf_nll_explicit(train_loader, var_model, args)
        # record E nL_n(w)
        nll_betas_explicit = np.append(nll_betas_explicit, sum(nllw_array_explicit)/len(nllw_array_explicit))

        # visualize EVI
        args.VItype = 'explicit'
        sampled_weights = sample_EVI(var_model, args)
        posterior_viz(train_loader, sampled_weights, args, beta_index, saveimgpath)

        # train implicit variational inference
        args.epsilon_dim = args.w_dim
        args.epsilon_mc = args.batchsize
        args.VItype = 'implicit'
        print('Begin training IVI')
        G = train_implicitVI(train_loader, valid_loader, args, mc, beta_index, saveimgpath)
        print('Finished training IVI')
        nllw_array_implicit = approxinf_nll_implicit(train_loader, G, args)
        nll_betas_implicit = np.append(nll_betas_implicit, sum(nllw_array_implicit)/len(nllw_array_implicit))

        # visualize IVI
        with torch.no_grad():
            eps = torch.randn(100, args.epsilon_dim)
            sampled_weights = G(eps)
            posterior_viz(train_loader, sampled_weights, args, beta_index, saveimgpath)
    
    return nll_betas_explicit, nll_betas_implicit

## Logistic regression
The binary target is modeled as $y = bernoulli(p)$, where $$p = \exp(w^T x + b)/(1+\exp(w^T x + b))$$ with $x$ is standard Gaussian. 
We set the true parameters to $w_0 = (0.5, 1)$ and $b = 0.0$.

In [ ]:
args.dataset = 'logistic_synthetic'
args.network = 'logistic'
args.bias = False
args.input_dim = args.w_dim
args.output_dim = 1

# Let's generate some data according to this model
args.w_0 = torch.Tensor([[0.5], [1]])
args.b = torch.tensor([0.0])
X = torch.randn(2 * args.syntheticsamplesize, args.input_dim)
affine = torch.mm(X, args.w_0) + args.b
m = torch.distributions.bernoulli.Bernoulli(torch.sigmoid(affine))
y = m.sample()

Let's first visualize the data.

In [ ]:
plt.plot(affine.squeeze(dim=1).detach().numpy(), y.detach().numpy(), '.g')
plt.plot(affine.squeeze(dim=1).detach().numpy(), torch.sigmoid(affine).detach().numpy(), '.r')
plt.xlabel('w^t x + b')
plt.ylabel('probs and binary response')
plt.show()

Next let's train explicit and implicit variational inference.

In [ ]:
nll_betas_explicit, nll_betas_implicit = train(args)


To see how well the variational infernece methods can estimate the RLCT, 
we plot $1/\beta$ versus the VI estimate of $E_w^\beta nL_n(w)$. 
The true RLCT should be the slope of this scatterplot.

In [ ]:
print('Explicit VI: RLCT estimate')
lsfit_lambda(nll_betas_explicit, args, saveimgpath=None)
print('Implicit VI: RLCT estimate')
lsfit_lambda(nll_betas_implicit, args, saveimgpath=None)


## Tanh regression
The univariate continuous target is modeled as $y = N(f(x,a,b),1)$,
where $$f(x,a,b) = \sum_{h=1}^H a_h \tanh(b_hx)$$ with $x \tilde uniform(0,1)$.
We will consider a two dimensional model where $H=1$ and $(a,b)=(0,0)$.
Note that this is a singular model where $\{a,b: K(a,b) = 0\} = \{a,b \in \mathbb R: ab = 0\}$

In [ ]:
args.dataset = 'tanh_synthetic'
args.network = 'tanh'
args.bias = False
args.H = 1
args.a_params = torch.zeros([1, args.H], dtype=torch.float32)
args.b_params = torch.zeros([args.H, 1], dtype=torch.float32)
args.input_dim = 1
args.output_dim = 1

m = Uniform(torch.tensor([0.0]), torch.tensor([1.0]))
X = m.sample(torch.Size([2 * args.syntheticsamplesize]))
mean = torch.matmul(torch.tanh(torch.matmul(X, args.a_params)), args.b_params)
y_rv = Normal(mean, 1)
y = y_rv.sample()

Let's first visualize the data.

In [ ]:
plt.plot(X.detach().numpy(), y.detach().numpy(), '.g')
plt.title('synthetic tanh regression data: x versus y')
plt.show()

Next let's train explicit and implicit variational inference.

In [ ]:
nll_betas_explicit, nll_betas_implicit = train(args)


To see how well the variational infernece methods can estimate the RLCT,
we plot $1/\beta$ versus the VI estimate of $E_w^\beta nL_n(w)$.
The true RLCT should be the slope of this scatterplot.

In [ ]:
print('Explicit VI: RLCT estimate')
lsfit_lambda(nll_betas_explicit, args, saveimgpath=None)
print('Implicit VI: RLCT estimate')
lsfit_lambda(nll_betas_implicit, args, saveimgpath=None)


## Reduced rank regression
The univariate continuous target is modeled as $y = N(f(x,A,B),1)$,
where $$f(x,A,B) = BAx$$ with $x$ standard Gaussian.
Considering a two dimensional model where we set the two matrices each to the scalar $1$.
Note that this is a singular model where we have $\{a,b: K(a,b) = 0\} = \{a,b \in \mathbb R: ab = 1\}$

In [ ]:
args.dataset = 'reducedrank_synthetic'
args.network = 'reducedrank'
args.bias = False
args.a_params = torch.Tensor([1.0]).reshape(1, 1)
args.b_params = torch.Tensor([1.0]).reshape(1, 1)
args.input_dim = 1
args.output_dim = 1
args.H = 1

# Let's generate some data according to this model
m = MultivariateNormal(torch.zeros(args.input_dim), torch.eye(
    args.input_dim))  # the input_dim=output_dim + 3, output_dim = H (the number of hidden units)
X = m.sample(torch.Size([2 * args.syntheticsamplesize]))
mean = torch.matmul(torch.matmul(X, args.a_params), args.b_params)
y_rv = MultivariateNormal(mean, torch.eye(args.output_dim))
y = y_rv.sample()

Let's first visualize the data.

In [ ]:
plt.plot(X.detach().numpy(), y.detach().numpy(), '.g')
plt.title('synthetic reduced rank regression data: x versus y')
plt.show()

Next let's train explicit and implicit variational inference.

In [ ]:
nll_betas_explicit, nll_betas_implicit = train(args)


To see how well the variational infernece methods can estimate the RLCT,
we plot $1/\beta$ versus the VI estimate of $E_w^\beta nL_n(w)$.
The true RLCT should be the slope of this scatterplot.

In [ ]:
print('Explicit VI: RLCT estimate')
evi_robust, evi_ols = lsfit_lambda(nll_betas_explicit, args, saveimgpath=None)
print('Implicit VI: RLCT estimate')
ivi_robust, ivi_ols = lsfit_lambda(nll_betas_implicit, args, saveimgpath=None)



